In [2]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime
from PIL import Image




In [6]:
# Website that hosts the data.
url = "https://cdn.star.nesdis.noaa.gov/GOES16/ABI/SECTOR/ne/GEOCOLOR/"


In [17]:

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

print("Accessed: ", datetime.now())
results = soup.find_all('a')
for link in tqdm(results):
    href = link.get('href')
    if href == "../":
        continue
    file_url = url + href

    # Get the file
    file_response = requests.get(file_url)

    # Skip downloading the gifs
    if ".jpg" not in file_url:
        continue
    if os.path.exists(f"../GEOCOLOR_IMAGES/{href}"):
        continue
    if file_response.status_code == 200:
        # Save the file to GEOCOLOR_IMAGES
        with open(f"../GEOCOLOR_IMAGES/{href}", 'wb') as file:
            file.write(file_response.content)


Accessed:  2025-02-07 17:41:37.371417


100%|██████████| 11683/11683 [2:03:23<00:00,  1.58it/s]  


In [3]:
# Convert to png


input_dir = "../GEOCOLOR_IMAGES_JPG"
base_output_dir = "../GEOCOLOR_IMAGES"

os.makedirs(base_output_dir, exist_ok=True)
os.makedirs(base_output_dir + "/300", exist_ok=True)
os.makedirs(base_output_dir + "/600", exist_ok=True)
os.makedirs(base_output_dir + "/1200", exist_ok=True)
os.makedirs(base_output_dir + "/2400", exist_ok=True)

for filename in tqdm(os.listdir(input_dir)):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        img = Image.open(os.path.join(input_dir, filename))
        if "300" in filename:
            output_dir =base_output_dir + "/300"
        elif "600" in filename:
            output_dir =base_output_dir + "/600"
        elif "1200" in filename:
            output_dir =base_output_dir + "/1200"
        elif "2400" in filename:
            output_dir =base_output_dir + "/2400"
       
        width, height = img.size
        new_height = int(height * 0.9)  # Remove bottom 10%  # This is where the text and watermark is
        img = img.crop((0, 0, width, new_height))

        img.save(os.path.join(output_dir, filename.replace(".jpg", ".png")))
         

100%|██████████| 11530/11530 [21:14<00:00,  9.05it/s]  
